# PCR DATA ANALYSIS: ELSA 🧬🧬🧬

### Halo sobat PCR 👋

### This is a fully automatic script to analyse the raw qPCR export file from Elsa (Kogene) and output the summary of the qPCR run into an excel file:
* Filters out samples when the Ct value of E and RDRP are both 0, *except for samples from repeated extraction (suffix -a)*
* Includes the sample and flag them when the Ct value of RNAse P is higher than 35
* Includes extraction controls and PCR controls when there are any Ct values
* Averages the Ct value of positive control at the bottom of the summary table

### Please make sure that:
* Positive and PCR controls (NTC) are properly named (e.g. POS01, POS02, NTC01, NTC02, etc.)
* No duplicate Sample ID (else it will cause error in the pivoting step)
* <span style="color:red">**THERE IS NO BATCH WHERE THE SAMPLES ARE NOT 15. Please use the alternative version of the script in this case.**</span> (Technically, this script can still be used with more/less than 15 samples, as long as the extraction control is at number 15)

### Please let Moreno know if there is any bug, cheers! 🍻

## DATA CLEANING



### 1. Import data


Input file name at:
file_directory = '<span style="color:red">**file name**</span>.xlsx'
> file_directory = '20210207 Covid Batch 7230 31 32 33-20210207143122.xlsx'
    
The full file path needs to be specified if the file is not in the same folder as this Jupyter Notebook file
> file_directory = '/Users/morenozan/Downloads/20210207 Covid Batch 7230 31 32 33-20210207143122.xlsx'

How to copy pathname:
>For Windows, hold shift -> right click -> copy as path -- **backslashes needs to be changed to regular slashes**

>For macOS, hold option -> right click -> copy file as pathname, or **CMD+Option+C**

In [ ]:
# import data

import pandas as pd
import numpy as np
import os

# copy paste the file name here

file_directory = '<file name here, include xlsx>'


# import the raw export file from Elsa into pandas DataFrame

df = pd.read_excel(file_directory)


# replace '-' to 0

df['Ct'].replace(['-'],0.00,inplace=True)
df

### 2. Exclude positive control, extraction control, and NTC

In [ ]:
filtered_df = df[(df['Sample ID'].str.contains('NTC')==False) & (df['Sample ID'].str.contains('POS')==False) & (df['Sample ID'].str.endswith('15')==False)]
filtered_df

### 3. Pivot E, RDRP, and RNAse P into their own columns, and then remove rows if E and RDRP are both equal to 0

Except for data with RNAse P higher than 35 and data from repeated extraction (with -A suffix), they will be kept

In [ ]:
# pivot the DataFrame (E, RDRP, and RNAse P became the columns)

df1 = filtered_df[['Sample ID','Gene','Ct']]
df1piv = df1.pivot(index = 'Sample ID', columns = 'Gene', values = 'Ct')


# exclude rows when the Ct value of both E and RDRP equals to 0 
# retains rows when RNAse P is higher than 35 and retains rows from repeated extraction

df2 = df1piv[((df1piv['E '] != 0)|(df1piv['RDRP'] != 0))|(df1piv['RNAse P'] > 35)|(df1piv['RNAse P'] == 0)|df1piv.index.str.endswith('A')][['E ','RDRP','RNAse P']]
df2

Optional: you can run the script below if you want to crosscheck with the full pivoted table (remove the # and run)

In [ ]:
#df1piv

## DATA ANALYSIS

### 1. Add POS/NEG description and RNAse P status

In [ ]:
# describe PCR result into a new column 'POS/NEG'

conditionsPN = [
    #invalid
    
    ((df2['RNAse P'] == 0)), # no Ct value for RNAse P, invalid
    
    
    #negative
    
    ((df2['E '] == 0) & (df2['RDRP'] == 0)), # E and RDRP = 0, negative  
    
    ((df2['E '] > 38) & (df2['RDRP'] > 38)), # E and RDRP both more than 38, negative
    
    ((df2['E '] == 0) & (df2['RDRP'] > 38)), # E = 0 and RDRP more than 38, negative
    
    ((df2['E '] > 38) & (df2['RDRP'] == 0)), # E more than 38 and RDRP = 0, negative
    
    
    #inconclusive
    
    ((df2['E '] < 38) & (df2['RDRP'] > 38)), # E less than 38, RDRP more than 38, inconclusive
    
    ((df2['E '] > 38) & (df2['RDRP'] < 38)),  # E more than 38, RDRP less than 38, inconclusive 
    
    ((df2['E '] == 0) & (df2['RDRP'] < 38)), # E = 0, RDRP less than 38, inconclusive
    
    ((df2['E '] < 38) & (df2['RDRP'] == 0)),  # E less than 38, RDRP = 0, inconclusive 
    
    
    #positive
    
    ((df2['E '] < 38) & (df2['RDRP'] < 38)) # E dan RDRP both less than 38, positive
]

valuesPN = ['Invalid','Negative','Negative','Negative','Negative','Inconclusive','Inconclusive','Inconclusive','Inconclusive','Positive']
df2['POS/NEG'] = np.select(conditionsPN,valuesPN)


# describe HRP status into a new column 'Aman?'

conditionsHRP = [
    ((df2['RNAse P'] == 0)), # no Ct value for HRP, 'Gk naik'
    
    ((df2['RNAse P'] > 35)), # High HRP Ct: 'Di atas 35'
    
    ((df2['RNAse P'] <=35))  # Normal HRP, blank
]

valuesHRP = ['Gk naik','Di atas 35','']
df2['Aman?'] = np.select(conditionsHRP,valuesHRP)
df2

### 2. Describe extraction and PCR (NTC) control status into a new DataFrame
# <span style="color:red">!!! Extraction control at .15; please use the alternative script if this is not the case !!!</span>

In [ ]:
# new DataFrame for EC and NTC

dfcekNTC = df[df['Sample ID'].str.contains('NTC')][['Sample ID','Gene','Ct']]
dfcekNTCpiv = dfcekNTC.pivot(index = 'Sample ID',columns = 'Gene',values = 'Ct')
dfcekEC = df[df['Sample ID'].str.endswith('15')][['Sample ID','Gene','Ct']]
dfcekECpiv = dfcekEC.pivot(index = 'Sample ID',columns = 'Gene',values = 'Ct')


# make column: 'Aman?' --> 'Naik' if HRP, E, and RDRP not equal to 0
#                      --> 'Aman' if all equal to 0

dfcekNTCpiv['Aman?'] = np.where((dfcekNTCpiv['RNAse P'] == 0) & (dfcekNTCpiv['E '] == 0) & (dfcekNTCpiv['RDRP'] == 0),'Aman','Naik')
dfcekECpiv['Aman?'] = np.where((dfcekECpiv['RNAse P'] == 0) & (dfcekECpiv['E '] == 0) & (dfcekECpiv['RDRP'] == 0),'Aman','Naik')


# new combined DataFrame

dfcek = pd.concat([dfcekECpiv,dfcekNTCpiv])


# filter only for those where the status is Naik

dfNaik = dfcek[dfcek['Aman?']=='Naik']


dfcek

### 3. Mean Ct for Positive Control

In [ ]:
# make a new pivot DataFrame (column Sample Id, E, RDRP, RNAse P) for positive control rows

dfPos = df[df['Sample ID'].str.contains('POS')][['Sample ID','Gene','Ct']]
dfPosPivot = dfPos.pivot(index = 'Sample ID', columns = 'Gene', values = 'Ct')

# make a new DataFrame for the mean of the Ct value of each gene in the positive control
dfPosMean = pd.DataFrame({'E ': dfPosPivot[['E ']].mean().at['E '],
                          'RDRP': dfPosPivot[['RDRP']].mean().at['RDRP'],
                          'RNAse P': dfPosPivot[['RNAse P']].mean().at['RNAse P']
    
            },index=['Positive Control'])
dfPosMean

## SUMMARY

### New combined DataFrame from the samples, extraction and PCR controls, and positive control
Extraction control dan NTC shows up if any of the Ct value is not 0

In [ ]:
dfOut = pd.concat([df2,dfNaik]).sort_index()
dfOutput = pd.concat([dfOut,dfPosMean]).fillna('')
dfOutput

### Export to an Excel file

In [ ]:
# filename for the output file

outputFilename = 'ANALYSED_' + os.path.basename(os.path.splitext(file_directory)[0])

# export dfOutput to a new excel file

dfOutput.to_excel(outputFilename + '.xlsx',index=True,header=True)